## 保存和加载模型

三个核心功能：
- torch.save: 将序列化对象保存到磁盘。
- torch.load: 将对象文件反序列化到内存，还有助于设备加载数据。
- torch.nn.Module.load_state_dict: 使用反序列化函数state_dict来加载模型的参数字典

#### 1. 什么是状态字典：state_dict ?

在PyTorch中，torch.nn.Module模型的可学习参数，也就是权重和偏差，包含在模型的参数中。
使用model.parameters()可以进行访问。

state_dict是python字典对象。方便保存 更新 修改和恢复。
将每一层映射到其参数张量。
（ps 只有具有学习参数的层的模型才有这个）

目标优化torch.optim也有state_dict属性，包含有关优化器的状态信息，以及使用的超参数。
示例：

In [4]:
import torch
import torch.nn as nn

class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 25, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 25)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
# 初始化模型
model = TheModelClass()
    
# 初始化优化器
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
# 打印模型的状态字典
print("Model's state_dict: ")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())
        
# 打印优化器的状态字典
print("Optimizer's state_dict: ")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict: 
conv1.weight 	 torch.Size([6, 3, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([16, 6, 5, 5])
conv2.bias 	 torch.Size([16])
fc1.weight 	 torch.Size([120, 400])
fc1.bias 	 torch.Size([120])
fc2.weight 	 torch.Size([84, 120])
fc2.bias 	 torch.Size([84])
fc3.weight 	 torch.Size([10, 84])
fc3.bias 	 torch.Size([10])
Optimizer's state_dict: 
state 	 {}
param_groups 	 [{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [140671936644032, 140669854762880, 140669854762240, 140669854762560, 140671945297504, 140672005680416, 140672005368640, 140669859754784, 140672005306480, 140671945320720]}]


#### 2. 保存和加载推理模型

##### 2.1 保存/加载state_dict（推荐使用）

- 保存

torch.save(model.state_dict(), PATH)

- 加载

model = TheModelClass(*args, ** kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()

当保存好模型用来推断的时候，只需要保存模型学习到的参数： torch.save()函数来保存模型的state_dict。
这样会给模型恢复提供最大的灵活性。

.pt .pth 作为模型文件的扩展名

注意，在运行推理之前，必须调用model.eval()去设置dropout和batch normalization层作为评估模式。
如果不这么做，可能导致模型推断的结果不一样。

另外，load_state_dict()函数只接受字典对象，而不是保存对象的路径。
所以在调用这个函数之前，必须反序列化保存的state_dict。


##### 2.2 保存/加载完整模型

- 保存
torch.save(model, PATH)
- 加载
model = torch.load(PATH)
model.eval()

以Python pickle 模块的方式来保存模型。

缺点是序列化数据受限于某些特殊种类的类而需要确切的字典结构。
因为pickle无法保存模型类本身。
而是保存包含类的文件的路径。所以自己的代码可能会被打断。

#### 3. 加载和保存Checkpoint用于推理/继续训练

- 保存

torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optmizer.state_dict(),
    'loss': loss,
    ...
}, PATH)


- 加载

model = TheModelClass(* args, ** kwargs)
optimizer - TheOptimizerClass(*args, ** kwargs)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state+dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
或者
model.train()

当保存成Checkpoint的时候，保存的不仅仅是模型的state_dict。
因为优化器的state_dict也很重要，因为它包含作为模型训练更新的缓冲区和参数。
其实也可以保存其他项目。比如最新记录的训练损失，外部的torch.nn.Embedding层等等。

要保存多个组件的时候，在字典中组织他们并使用torch.save()来序列化字典。
.tar 文件扩展名


#### 4. 在一个文件中保存多个模型

torch.save({
    'modelA_state_dict' : modelA.state_dict(),
    'modelB_state_dict' : modelB.state_dict(),
    ...
})

加载的时候也是类似Checkpoint

剩下的部分其实都很类似。
最后的一点还有在cpu和gpu上反复横跳的操作。